# Module 2 Final Project: Northwind Hypothesis Testing

## Setup

Importing packages and creating the engine and session for me to explore the Northwind database

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
%matplotlib inline

In [2]:
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session, sessionmaker

engine = create_engine("sqlite:///Northwind_small.sqlite", echo=True)

Session = sessionmaker(bind=engine)
session = Session()

inspector = inspect(engine)
print(inspector.default_schema_name)

2019-03-19 09:45:29,875 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-03-19 09:45:29,877 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:29,878 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-03-19 09:45:29,879 INFO sqlalchemy.engine.base.Engine ()
None


Creating an ORM to easily pull data from the necessary tables

In [3]:
from sqlalchemy import MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import relationship

metadata = MetaData()
metadata.reflect(engine)

Base = automap_base(metadata=metadata)

Base.prepare()

Order = Base.classes.Order
OrderDetail = Base.classes.OrderDetail
Product = Base.classes.Product
Customer = Base.classes.Customer
Category = Base.classes.Category
Employee = Base.classes.Employee

2019-03-19 09:45:29,954 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-03-19 09:45:29,956 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:29,961 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Category")
2019-03-19 09:45:29,962 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:29,966 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Category' AND type = 'table'
2019-03-19 09:45:29,967 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:29,971 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Category")
2019-03-19 09:45:29,972 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:29,974 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Category' AND type = 'table'
2019-03-19 09:45:29,975 INFO sqlalchemy.engine.base.Engin

2019-03-19 09:45:30,097 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("EmployeeTerritory")
2019-03-19 09:45:30,098 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,099 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'EmployeeTerritory' AND type = 'table'
2019-03-19 09:45:30,100 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,101 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("EmployeeTerritory")
2019-03-19 09:45:30,102 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,103 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("EmployeeTerritory")
2019-03-19 09:45:30,104 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,106 INFO sqlalchemy.engine.base.Engine PRAGMA index_info("sqlite_autoindex_EmployeeTerritory_1")
2019-03-19 09:45:30,107 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,108 INFO sqlalchemy.engine.base.Engine SELECT sql 

2019-03-19 09:45:30,220 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Supplier")
2019-03-19 09:45:30,221 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,225 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Supplier' AND type = 'table'
2019-03-19 09:45:30,225 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,227 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Supplier")
2019-03-19 09:45:30,228 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,229 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Supplier' AND type = 'table'
2019-03-19 09:45:30,230 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,232 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("Supplier")
2019-03-19 09:45:30,233 INFO sqlalchemy.engine.base.Engine ()
2019-03-19 09:45:30,234 INFO 

## Discount Effects Analysis

### Refining the Question

The original question as posed was: "Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount?"

I see a few different ways to rephase this question:

1. On average across purchases of products that are sometimes discounted, do orders that include that product include a significantly higher quantity of it when that product has a discount applied versus when it does not? Does the difference vary for different discount sizes?
2. On average across purchases of products that are sometimes discounted, are customers more likely to order products that are sometimes discounted versus products that are never discounted? Does the difference vary for different discount sizes?
3. On average across orders, do orders that contain discounted items tend to include more total items than orders that do not include discounted items? Does the difference vary for different discount sizes?

I have chosen to pursue question #1 because that seems like the most plausible scenario based on my understanding of purchasing behavior.

### Data Extraction

Answering this query will require columns from Order, and OrderDetail, which I will pull into a DataFrame with one row per OrderDetail

In [4]:
query1 = session.query(Order, OrderDetail).\
                filter(OrderDetail.OrderId == Order.Id).\
                order_by(Order.Id).\
                all()

df1 = pd.DataFrame()
for o, od in query1:
    q1_df_row = pd.DataFrame({'Order_Id': o.Id, 'Product_Id': od.ProductId, 
                             'Product_Quantity': od.Quantity, 'Product_Discount': od.Discount}, index = [0])
    df1 = df1.append(q1_df_row, ignore_index= True)

2019-03-19 09:45:30,312 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-03-19 09:45:30,315 INFO sqlalchemy.engine.base.Engine SELECT "Order"."Id" AS "Order_Id", "Order"."CustomerId" AS "Order_CustomerId", "Order"."EmployeeId" AS "Order_EmployeeId", "Order"."OrderDate" AS "Order_OrderDate", "Order"."RequiredDate" AS "Order_RequiredDate", "Order"."ShippedDate" AS "Order_ShippedDate", "Order"."ShipVia" AS "Order_ShipVia", "Order"."Freight" AS "Order_Freight", "Order"."ShipName" AS "Order_ShipName", "Order"."ShipAddress" AS "Order_ShipAddress", "Order"."ShipCity" AS "Order_ShipCity", "Order"."ShipRegion" AS "Order_ShipRegion", "Order"."ShipPostalCode" AS "Order_ShipPostalCode", "Order"."ShipCountry" AS "Order_ShipCountry", "OrderDetail"."Id" AS "OrderDetail_Id", "OrderDetail"."OrderId" AS "OrderDetail_OrderId", "OrderDetail"."ProductId" AS "OrderDetail_ProductId", "OrderDetail"."UnitPrice" AS "OrderDetail_UnitPrice", "OrderDetail"."Quantity" AS "OrderDetail_Quantity", "OrderDetail

C:\Users\david_000\Anaconda3\lib\site-packages\sqlalchemy\sql\sqltypes.py:603: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


### Data Exploration & Cleaning

In [5]:
print(df1.info())
print(df1.describe())
df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 4 columns):
Order_Id            2155 non-null int64
Product_Id          2155 non-null int64
Product_Quantity    2155 non-null int64
Product_Discount    2155 non-null float64
dtypes: float64(1), int64(3)
memory usage: 67.4 KB
None
           Order_Id   Product_Id  Product_Quantity  Product_Discount
count   2155.000000  2155.000000       2155.000000       2155.000000
mean   10659.375870    40.793039         23.812993          0.056167
std      241.378032    22.159019         19.022047          0.083450
min    10248.000000     1.000000          1.000000          0.000000
25%    10451.000000    22.000000         10.000000          0.000000
50%    10657.000000    41.000000         20.000000          0.000000
75%    10862.500000    60.000000         30.000000          0.100000
max    11077.000000    77.000000        130.000000          0.250000


,Order_Id,Product_Id,Product_Quantity,Product_Discount
0,10248,11,12,0.0
1,10248,42,10,0.0
2,10248,72,5,0.0
3,10249,14,9,0.0
4,10249,51,40,0.0


In [6]:
#Number of Orders
print(len(df1['Order_Id'].unique()))
df1['Order_Id'] = df1['Order_Id'].astype('category')

#Number of Products
print(len(df1['Product_Id'].unique()))
df1['Product_Id'] = df1['Product_Id'].astype('category')

#Product discount amounts
print(df1['Product_Discount'].value_counts())
#Lump all small discounts into 0.05 category and recast as categorical
df1['Product_Discount'] = df1['Product_Discount'].apply(lambda x: int(5) if (x < 0.1) & (x > 0) else int(x*100))
df1['Product_Discount'] = df1['Product_Discount'].astype('category')
print(df1['Product_Discount'].value_counts())

830
77
0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
0.03       3
0.02       2
0.01       1
0.04       1
0.06       1
Name: Product_Discount, dtype: int64
0     1317
5      193
10     173
20     161
15     157
25     154
Name: Product_Discount, dtype: int64


In [7]:
#Product discount cleaning
df1 = pd.get_dummies(df1, columns=['Product_Discount'])
df1['Has_Discount'] = (1 - df1['Product_Discount_0'])
df1.head()

,Order_Id,Product_Id,Product_Quantity,Product_Discount_0,Product_Discount_5,Product_Discount_10,Product_Discount_15,Product_Discount_20,Product_Discount_25,Has_Discount
0,10248,11,12,1,0,0,0,0,0,0
1,10248,42,10,1,0,0,0,0,0,0
2,10248,72,5,1,0,0,0,0,0,0
3,10249,14,9,1,0,0,0,0,0,0
4,10249,51,40,1,0,0,0,0,0,0


Next, I have to check to see which products are sometimes discounted, meaning that in some orders they have a discount and in some orders they don't.

In [8]:
#Create DataFrame indicating whether products are ever discounted or not
disc_df = df1[['Product_Id', 'Has_Discount']]
disc_df = disc_df.drop_duplicates()
disc_df = disc_df.sort_values(['Product_Id', 'Has_Discount'], axis = 0)
disc_df.shape

(154, 2)

Since there are 77 products, and 77*2 = 154, this means that every product is sometimes discounted and sometimes not discounted. This makes life a lot easier, because I can analyze all products in all orders.

In [9]:
product_list = list(df1['Product_Id'].drop_duplicates())

Next, I look at the relationship between product quantities for products sold at a discount versus undiscounted products.

In [10]:
df1.groupby(by = 'Has_Discount').mean()['Product_Quantity']

Has_Discount
0    21.715262
1    27.109785
Name: Product_Quantity, dtype: float64

### Analysis

This first analysis adresses the first part of the question: On average across purchases of products that are sometimes discounted, do orders that include that product include a significantly higher quantity of it when that product has a discount applied versus when it does not?

To do this, I will use two regression models. The first model is a simple linear regression. In this model, the target is 'Product_Quantity', the quantity of the ordered product. The feature of interest is a dummy variable that indicated whether the product was discounted: 'Has_Discount'.

In [11]:
#Fixed Effects Model
f = 'Product_Quantity~Has_Discount'
model = ols(formula=f, data=df1).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Product_Quantity   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     41.97
Date:                Tue, 19 Mar 2019   Prob (F-statistic):           1.14e-10
Time:                        09:45:34   Log-Likelihood:                -9384.3
No. Observations:                2155   AIC:                         1.877e+04
Df Residuals:                    2153   BIC:                         1.878e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       21.7153      0.519     41.821      0.000      20.697      22.734
Has_Discount     5.3945      0.833      6.479      0.000       3.762       7.027
==============================================================================
Omnibus:                      781.498   Durbin-Watson:                   1.629
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3015.153
Skew:                           1.761   Prob(JB):                         0.00
Kurtosis:                       7.602   Cond. No.                         2.44
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We see in this model that the coefficient associated with the 'Has_Discount' variable is 5.3945, with a P-value below 0.001. The null hypothesis associated with this coefficient is that the true value of the coefficient is zero, meaning that there is no difference in product quantity ordered associated with discounts. The alternative hypothesis is that the coefficient is not zero, meaning that there is a difference in purchase quantity associated with having a discount. This alternative hypothesis means that I am conducting a two-tailed test. 

Since our P-value is below 0.05, a common benchmark, we reject the null hypothesis and conclude that the presence of a discount is associated with a significantly different number of items purchased. In this case, we estimate that a discount is associated with an average increase of 5.4 items purchased.

Next, I will use a fixed effects regression model.  I include dummy variables for each product in the model as well - this adjusts for the fact that some people tend to order larger quantities of some products than others, regardless of whether they are discounted. This allows for the following interpretation of the coefficient on the 'Has_Discount' variable: it represents a weighted average across all products of the difference in quantity purchased associated with the presence of a discount. The weighting in this average is the number of purchases including each item.

In [12]:
#Fixed Effects Model
fe_df = pd.get_dummies(df1, columns=['Product_Id'], drop_first=True)
prod_id_col_list = fe_df.columns[9:]
prod_id_string = '+'.join(prod_id_col_list)
f = 'Product_Quantity~Has_Discount + ' + prod_id_string
model = ols(formula=f, data=fe_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Product_Quantity   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.441
Date:                Tue, 19 Mar 2019   Prob (F-statistic):            0.00805
Time:                        09:45:35   Log-Likelihood:                -9349.0
No. Observations:                2155   AIC:                         1.885e+04
Df Residuals:                    2077   BIC:                         1.930e+04
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        19.4876      3.083      6.322      0.000      13.442      25.533
Has_Discount      5.4670      0.850      6.432      0.000       3.800       7.134
Product_Id_2      1.4289      4.182      0.342      0.733      -6.772       9.630
Product_Id_3      6.9346      6.254      1.109      0.268      -5.329      19.198
Product_Id_4      0.4289      5.214      0.082      0.934      -9.797      10.655
Product_Id_5      8.1256      6.708      1.211      0.226      -5.029      21.281
Product_Id_6      3.7734      6.250      0.604      0.546      -8.484      16.031
Product_Id_7      5.3146      4.655      1.142      0.254      -3.815      14.444
Product_Id_8      7.0251      6.064      1.158      0.247      -4.868      18.918
Product_Id_9     -3.7678      8.980     -0.420      0.675     -21.379      13.843
Product_Id_10     1.1749      4.492      0.262      0.794      -7.634       9.983
Product_Id_11    -2.7789      4.330     -0.642      0.521     -11.271       5.714
Product_Id_12     2.3504      5.901      0.398      0.690      -9.222      13.923
Product_Id_13     1.0106      4.276      0.236      0.813      -7.376       9.397
Product_Id_14    -3.6089      5.056     -0.714      0.475     -13.525       6.307
Product_Id_15    -0.9766      8.292     -0.118      0.906     -17.237      15.284
Product_Id_16     4.5184      4.203      1.075      0.283      -3.725      12.762
Product_Id_17     4.5807      4.359      1.051      0.293      -3.968      13.129
Product_Id_18    -2.1569      4.751     -0.454      0.650     -11.474       7.160
Product_Id_19    -2.1634      4.359     -0.496      0.620     -10.712       6.385
Product_Id_20    -2.3169      5.625     -0.412      0.680     -13.348       8.714
Product_Id_21     4.1807      4.302      0.972      0.331      -4.256      12.618
Product_Id_22     4.5886      5.905      0.777      0.437      -6.993      16.170
Product_Id_23     8.1457      5.216      1.562      0.119      -2.083      18.375
Product_Id_24     0.6417      4.045      0.159      0.874      -7.291       8.574
Product_Id_25    -4.8581      5.402     -0.899      0.369     -15.451       5.735
Product_Id_26     2.1644      4.529      0.478      0.633      -6.717      11.046
Product_Id_27    20.4605      7.002      2.922      0.004       6.729      34.192
Product_Id_28    -1.7503      4.492     -0.390      0.697     -10.560       7.059
Product_Id_29     1.6040      4.528      0.354      0.723      -7.277      10.485
Product_Id_30    -2.7544      4.528     -0.608      0.543     -11.635       6.126
Product_Id_31     5.1175      4.045      1.265      0.206      -2.816      13.051
Product_Id_32    -2.6033      5.756     -0.452      0.651     -13.891       8.685
Product_Id_33     2.2269      4.529      0.492      0.623      -6.655      11.108
Product_Id_34     5.1298      5.303      0.967   

We see in this model that the coefficient associated with the 'Has_Discount' variable is 5.4670, with a P-value below 0.001. The null hypothesis associated with this coefficient is that the true value of the coefficient is zero, meaning that there is no difference in product quantity ordered associated with discounts, after accounting for differences in purchase quantity across products. The alternative hypothesis is that the coefficient is not zero, meaning that there is a difference in purchase quantity associated with having a discount. This alternative hypothesis means that I am conducting a two-tailed test. 

Since our P-value is below 0.05, a common benchmark, we reject the null hypothesis and conclude that the presence of a discount is associated with a significantly different number of items purchased. In this case, we estimate that a discount is associated with an average increase of 5.5 items purchased.

This next analysis addreses the second part of the question: Does the difference vary for different discount sizes? 

The previous model simply tested for observed differences associated with discounts of any size. Now, I will treat discount amount as a categorical variable, and observe the size of the differences in product quantity associated with each discount category. 

In [13]:
discount_col_list = fe_df.columns[3:8]
discount_string = '+'.join(discount_col_list)
f = 'Product_Quantity~' + discount_string + ' + ' + prod_id_string
model = ols(formula=f, data=fe_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Product_Quantity   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.411
Date:                Tue, 19 Mar 2019   Prob (F-statistic):             0.0103
Time:                        09:45:35   Log-Likelihood:                -9347.3
No. Observations:                2155   AIC:                         1.886e+04
Df Residuals:                    2073   BIC:                         1.932e+04
Df Model:                          81                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              19.2122      3.088      6.221      0.000      13.156      25.269
Product_Discount_5      5.1916      1.483      3.500      0.000       2.283       8.101
Product_Discount_10     3.5410      1.553      2.280      0.023       0.495       6.587
Product_Discount_15     6.5307      1.626      4.016      0.000       3.342       9.719
Product_Discount_20     5.2729      1.607      3.281      0.001       2.121       8.425
Product_Discount_25     7.0919      1.636      4.335      0.000       3.884      10.300
Product_Id_2            1.4685      4.186      0.351      0.726      -6.740       9.677
Product_Id_3            7.5310      6.264      1.202      0.229      -4.754      19.816
Product_Id_4            0.9070      5.223      0.174      0.862      -9.336      11.150
Product_Id_5            8.4700      6.715      1.261      0.207      -4.698      21.638
Product_Id_6            3.9822      6.264      0.636      0.525      -8.303      16.268
Product_Id_7            5.6278      4.664      1.207      0.228      -3.520      14.775
Product_Id_8            7.1750      6.067      1.183      0.237      -4.722      19.072
Product_Id_9           -3.6449      8.987     -0.406      0.685     -21.269      13.979
Product_Id_10           1.4579      4.497      0.324      0.746      -7.361      10.277
Product_Id_11          -2.5207      4.335     -0.581      0.561     -11.022       5.981
Product_Id_12           2.4306      5.909      0.411      0.681      -9.157      14.018
Product_Id_13           1.3237      4.283      0.309      0.757      -7.076       9.723
Product_Id_14          -3.3602      5.060     -0.664      0.507     -13.283       6.562
Product_Id_15          -0.6094      8.310     -0.073      0.942     -16.907      15.688
Product_Id_16           4.7111      4.212      1.118      0.264      -3.550      12.972
Product_Id_17           4.8925      4.369      1.120      0.263      -3.676      13.461
Product_Id_18          -1.7771      4.758     -0.374      0.709     -11.107       7.553
Product_Id_19          -1.8394      4.367     -0.421      0.674     -10.403       6.725
Product_Id_20          -1.9417      5.639     -0.344      0.731     -13.001       9.118
Product_Id_21           4.6677      4.313      1.082      0.279      -3.790      13.125
Product_Id_22           4.7676      5.910      0.807      0.420      -6.823      16.358
Product_Id_23           8.4064      5.219      1.611      0.107      -1.830      18.642
Product_Id_24           0.8153      4.048      0.201      0.840      -7.123       8.753
Product_Id_25          -4.7382      5.407     -0.876      0.381     -15.342       5.865
Product_Id_26           2.2555      4.532      0.498      0.619      -6.633      11.144
Product_Id_27          20.5554      7.005      2.934      0.003       6.818      34.293
Pro

### Interpretation

In this model, we have five coefficients for the five different discount categories. Each of these coefficients has the following null hypothesis: that the difference in purchase quantities associated with a discount of that size is zero. The alternative hypothesis is that the difference is non-zero, again a two-tailed test. We see that the P-value for each coefficient is below 0.05, meaning that there does seem to be a significant difference in product quantity purchased for discounts of every level. 

The coefficients range from 3.5410 to 7.0919, but I do not fomarlly test these difference to see if there are differences in effect size among the different discount categories. Given that the standard errors for these coefficients are around 1.6, we would be unlikely to see many difference emerge using formal tests, using a heuristic of 2 standard errors as a typical difference that would be needed to reject a null hypothesis of differences between coefficients.

This implies that there is no clear difference in product purchasing behavior associated with discounts of different sizes. However, we can say that discounts of any size are associated with increased purchase quantity.

## Discount Effects By Product Category

### Refining the Question

It seems plausible that the potential impacts of offering discounts might vary by product category. This would be useful knowledge for deciding which products to offer discounts for. I will test this hypothesis by adding in interaction terms to the previous model.

### Data Extraction

Data extraction will be similar to that for the previous query, with additional joins to pull in category information.

In [14]:
query2 = session.query(Order, OrderDetail, Product, Category).\
                filter(OrderDetail.OrderId == Order.Id).\
                filter(OrderDetail.ProductId == Product.Id).\
                filter(Product.CategoryId == Category.Id).\
                all()
df2 = pd.DataFrame()

for o, od, p, c in query2:
    q2_df_row = pd.DataFrame({'Order_Id': o.Id, 'Product_Id': od.ProductId, 
                             'Product_Quantity': od.Quantity, 'Product_Discount': od.Discount, 'Product_Category': c.CategoryName}, index = [0])
    df2 = df2.append(q2_df_row, ignore_index= True)

2019-03-19 09:45:35,706 INFO sqlalchemy.engine.base.Engine SELECT "Order"."Id" AS "Order_Id", "Order"."CustomerId" AS "Order_CustomerId", "Order"."EmployeeId" AS "Order_EmployeeId", "Order"."OrderDate" AS "Order_OrderDate", "Order"."RequiredDate" AS "Order_RequiredDate", "Order"."ShippedDate" AS "Order_ShippedDate", "Order"."ShipVia" AS "Order_ShipVia", "Order"."Freight" AS "Order_Freight", "Order"."ShipName" AS "Order_ShipName", "Order"."ShipAddress" AS "Order_ShipAddress", "Order"."ShipCity" AS "Order_ShipCity", "Order"."ShipRegion" AS "Order_ShipRegion", "Order"."ShipPostalCode" AS "Order_ShipPostalCode", "Order"."ShipCountry" AS "Order_ShipCountry", "OrderDetail"."Id" AS "OrderDetail_Id", "OrderDetail"."OrderId" AS "OrderDetail_OrderId", "OrderDetail"."ProductId" AS "OrderDetail_ProductId", "OrderDetail"."UnitPrice" AS "OrderDetail_UnitPrice", "OrderDetail"."Quantity" AS "OrderDetail_Quantity", "OrderDetail"."Discount" AS "OrderDetail_Discount", "Product"."Id" AS "Product_Id", "Pro

### Data Exploration & Cleaning

This will also be similar to the previous analysis, with some added work for the new product category information.

In [15]:
print(df2.info())
print(df2.describe())
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 5 columns):
Order_Id            2155 non-null int64
Product_Id          2155 non-null int64
Product_Quantity    2155 non-null int64
Product_Discount    2155 non-null float64
Product_Category    2155 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 84.3+ KB
None
           Order_Id   Product_Id  Product_Quantity  Product_Discount
count   2155.000000  2155.000000       2155.000000       2155.000000
mean   10659.375870    40.793039         23.812993          0.056167
std      241.378032    22.159019         19.022047          0.083450
min    10248.000000     1.000000          1.000000          0.000000
25%    10451.000000    22.000000         10.000000          0.000000
50%    10657.000000    41.000000         20.000000          0.000000
75%    10862.500000    60.000000         30.000000          0.100000
max    11077.000000    77.000000        130.000000          0.250000


,Order_Id,Product_Id,Product_Quantity,Product_Discount,Product_Category
0,10248,11,12,0.0,Dairy Products
1,10248,42,10,0.0,Grains/Cereals
2,10248,72,5,0.0,Dairy Products
3,10249,14,9,0.0,Produce
4,10249,51,40,0.0,Produce


In [16]:
#Number of Orders
print(len(df2['Order_Id'].unique()))
df2['Order_Id'] = df2['Order_Id'].astype('category')

#Number of Products
print(len(df1['Product_Id'].unique()))
df2['Product_Id'] = df2['Product_Id'].astype('category')

#Product discount amounts
print(df2['Product_Discount'].value_counts())
#Lump all small discounts into 0.05 category and recast as categorical
df2['Product_Discount'] = df2['Product_Discount'].apply(lambda x: int(5) if (x < 0.1) & (x > 0) else int(x*100))
df2['Product_Discount'] = df2['Product_Discount'].astype('category')
print(df2['Product_Discount'].value_counts())

#Product Categories
print(df2['Product_Category'].value_counts())
df2['Product_Category'] = df2['Product_Category'].apply(lambda x:'Dairy' if x == 'Dairy Products' else x)
df2['Product_Category'] = df2['Product_Category'].apply(lambda x:'Grains' if x == 'Grains/Cereals' else x)
df2['Product_Category'] = df2['Product_Category'].apply(lambda x:'Meat' if x == 'Meat/Poultry' else x)
df2['Product_Category'] = df2['Product_Category'].astype('category')

830
77
0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
0.03       3
0.02       2
0.01       1
0.04       1
0.06       1
Name: Product_Discount, dtype: int64
0     1317
5      193
10     173
20     161
15     157
25     154
Name: Product_Discount, dtype: int64
Beverages         404
Dairy Products    366
Confections       334
Seafood           330
Condiments        216
Grains/Cereals    196
Meat/Poultry      173
Produce           136
Name: Product_Category, dtype: int64


In [17]:
#Product discount cleaning
df2 = pd.get_dummies(df2, columns=['Product_Discount'])
df2['Has_Discount'] = (1 - df2['Product_Discount_0'])
df2.head()

,Order_Id,Product_Id,Product_Quantity,Product_Category,Product_Discount_0,Product_Discount_5,Product_Discount_10,Product_Discount_15,Product_Discount_20,Product_Discount_25,Has_Discount
0,10248,11,12,Dairy,1,0,0,0,0,0,0
1,10248,42,10,Grains,1,0,0,0,0,0,0
2,10248,72,5,Dairy,1,0,0,0,0,0,0
3,10249,14,9,Produce,1,0,0,0,0,0,0
4,10249,51,40,Produce,1,0,0,0,0,0,0


### Analysis

This analysis will build on the previous fixed effects regression model by adding interaction terms. Based on the results of the previous analysis, I am comfortable using the 'Has_Discount' dummy variable in lieu of the categorical discount variable. I will interact the product category dummies with this variable to create a number of interaction terms.

In [18]:
#Create dummy variables for analysis
fe_df = pd.get_dummies(df2, columns=['Product_Category', 'Product_Id'], drop_first=False)
fe_df_col_list = list(fe_df.columns)

#Create interaction terms with product categories and discounts
prod_cat_col_list = [c for c in fe_df_col_list if ('Product_Category' in c)]

for cat in [c for c in fe_df_col_list if ('Product_Category' in c)]:
    print(cat)
    new_col = cat + '_X_Disc'
    fe_df[new_col] = fe_df[cat] * fe_df['Has_Discount']


#drop unneeded dummies - one for each categorical variable
fe_df.drop(columns = ['Product_Id_1','Product_Category_Beverages'], axis = 1, inplace = True)

#Fixed Effects Model with Interactions
fe_df_col_list = list(fe_df.columns)
prod_cat_int_col_list = [c for c in fe_df_col_list if ('_X_Disc' in c)]
prod_id_col_list = [c for c in fe_df_col_list if ('Product_Id' in c)]

cat_string = '+'.join(prod_cat_int_col_list)
prod_id_string = '+'.join(prod_id_col_list)
f = 'Product_Quantity~Has_Discount + ' + cat_string + ' + ' + prod_id_string
model = ols(formula=f, data=fe_df).fit()
model.summary()

Product_Category_Beverages
Product_Category_Condiments
Product_Category_Confections
Product_Category_Dairy
Product_Category_Grains
Product_Category_Meat
Product_Category_Produce
Product_Category_Seafood


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Product_Quantity   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.424
Date:                Tue, 19 Mar 2019   Prob (F-statistic):            0.00773
Time:                        09:45:40   Log-Likelihood:                -9344.5
No. Observations:                2155   AIC:                         1.886e+04
Df Residuals:                    2070   BIC:                         1.934e+04
Df Model:                          84                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              18.7611      3.169      5.921      0.000      12.547      24.975
Has_Discount                            4.7439      0.814      5.825      0.000       3.147       6.341
Product_Category_Beverages_X_Disc       2.4484      1.902      1.287      0.198      -1.282       6.179
Product_Category_Condiments_X_Disc      4.2258      2.500      1.691      0.091      -0.676       9.128
Product_Category_Confections_X_Disc    -2.6633      2.072     -1.286      0.199      -6.726       1.400
Product_Category_Dairy_X_Disc           2.2339      2.000      1.117      0.264      -1.688       6.156
Product_Category_Grains_X_Disc         -3.2458      2.710     -1.198      0.231      -8.560       2.068
Product_Category_Meat_X_Disc            3.2330      2.709      1.193      0.233      -2.079       8.545
Product_Category_Produce_X_Disc        -1.0709      3.170     -0.338      0.736      -7.287       5.146
Product_Category_Seafood_X_Disc        -0.4173      2.052     -0.203      0.839      -4.441       3.606
Product_Id_2                            1.1750      4.188      0.281      0.779      -7.038       9.388
Product_Id_3                            7.0773      6.317      1.120      0.263      -5.311      19.465
Product_Id_4                           -0.5960      5.444     -0.109      0.913     -11.272      10.080
Product_Id_5                            7.4510      6.840      1.089      0.276      -5.963      20.865
Product_Id_6                            3.3323      6.364      0.524      0.601      -9.148      15.813
Product_Id_7                            6.5360      4.820      1.356      0.175      -2.917      15.989
Product_Id_8                            6.4044      6.204      1.032      0.302      -5.761      18.570
Product_Id_9                           -4.5473      9.183     -0.495      0.621     -22.555      13.461
Product_Id_10                           2.2815      4.619      0.494      0.621      -6.776      11.340
Product_Id_11                          -2.5693      4.462     -0.576      0.565     -11.320       6.181
Product_Id_12                           2.3214      6.045      0.384      0.701      -9.533      14.176
Product_Id_13                           2.1077      4.407      0.478      0.632      -6.535      10.750
Product_Id_14                          -2.0670      5.358     -0.386      0.700     -12.576       8.442
Product_Id_15                          -1.4177      8.376     -0.169      0.866     -17.845      15.009
Product_Id_16                           7.0562      4.433      1.592      0.112      -1.638      15.750
Product_Id_17                           4.2218      4.611      0.916      0.360      -4.822      13.265
Product_Id_18                          -0.8813      4.928     -0.17

The coefficients on the interaction variables range from -3.2458 to 4.2258, all with P-values above 0.05. This means that we cannot reject the null hypothesis for each of these interaction terms - that the coefficients are equal to zero - in favor of the alternative hypothesis that the coefficients are non-zero - a two-tailed test. This means that we do not have evidence to claim that the magnitude of the difference in order quantity associated with discounts is different for different categories of products. 

The high standard errors on these coefficients and high condition number suggests that we have insufficient sample size to get precise estimates for these interaction term coefficients in the face of high multi-collinearity.

## Order Count by Region Match

### Refining the Question

Customers may be more satisfied if their orders are processed by an employee who works in their region. This could be because the employee is better able to communicate with or understand the needs of their customer. One indicator of customer satisfaction is whether the customer submits more orders. 

Here is the question framed in a more answerable way: On average across employees in each region, do customers from that region submit more orders than customers from other regions? If so, is this difference similar across regions?

### Data Extraction

This will require pulling information from the CustomerDemo table, so I will need to add a few joins to the query

In [30]:
query3 = session.query(Order, Employee, Customer).\
                filter(Customer.Id == Order.CustomerId).\
                filter(Employee.Id == Order.EmployeeId).\
                all()

df3 = pd.DataFrame()
for o, e, c in query3:
    q3_df_row = pd.DataFrame({'Order_Id': o.Id, 'Customer_Id': c.Id, 'Employee_Id': e.Id,
                             'Customer_Region': c.Region, 'Employee_Region': e.Region}, index = [0])
    df3 = df3.append(q3_df_row, ignore_index= True)

2019-03-19 10:13:34,860 INFO sqlalchemy.engine.base.Engine SELECT "Order"."Id" AS "Order_Id", "Order"."CustomerId" AS "Order_CustomerId", "Order"."EmployeeId" AS "Order_EmployeeId", "Order"."OrderDate" AS "Order_OrderDate", "Order"."RequiredDate" AS "Order_RequiredDate", "Order"."ShippedDate" AS "Order_ShippedDate", "Order"."ShipVia" AS "Order_ShipVia", "Order"."Freight" AS "Order_Freight", "Order"."ShipName" AS "Order_ShipName", "Order"."ShipAddress" AS "Order_ShipAddress", "Order"."ShipCity" AS "Order_ShipCity", "Order"."ShipRegion" AS "Order_ShipRegion", "Order"."ShipPostalCode" AS "Order_ShipPostalCode", "Order"."ShipCountry" AS "Order_ShipCountry", "Employee"."Id" AS "Employee_Id", "Employee"."LastName" AS "Employee_LastName", "Employee"."FirstName" AS "Employee_FirstName", "Employee"."Title" AS "Employee_Title", "Employee"."TitleOfCourtesy" AS "Employee_TitleOfCourtesy", "Employee"."BirthDate" AS "Employee_BirthDate", "Employee"."HireDate" AS "Employee_HireDate", "Employee"."Addr

### Data Exploration & Cleaning

In [31]:
#Explore counts of Region & Employee IDs
print(df3['Customer_Region'].value_counts())
print(df3['Employee_Region'].value_counts())
print(df3['Employee_Id'].value_counts())
print(df3[['Employee_Id', 'Employee_Region']].drop_duplicates())

#Create Region Match Variable
region_match_list = [df3['Customer_Region'][i] == df3['Employee_Region'][i] for i in range(0,df3.shape[0])]
df3['Region_Match'] = pd.Series(region_match_list)
print(df3['Region_Match'].value_counts())

#Create order count variable
order_count = df3.groupby('Customer_Id').count()['Order_Id']
df3 = df3.merge(order_count.to_frame('Cust_Ord_Count'), left_on='Customer_Id', right_index=True)
print(df3['Cust_Ord_Count'].value_counts())

df3.head(10)

Western Europe     272
North America      152
South America      127
British Isles       75
Southern Europe     64
Northern Europe     55
Scandinavia         28
Central America     21
Eastern Europe       7
Name: Customer_Region, dtype: int64
North America    590
British Isles    211
Name: Employee_Region, dtype: int64
4    152
3    123
1    121
8    100
2     94
6     64
7     63
9     43
5     41
Name: Employee_Id, dtype: int64
    Employee_Id Employee_Region
0             5   British Isles
1             6   British Isles
2             4   North America
3             3   North America
7             9   British Isles
10            1   North America
14            8   North America
17            2   North America
41            7   British Isles
False    661
True     140
Name: Region_Match, dtype: int64
10    110
14     84
18     54
6      48
9      45
5      45
15     45
11     44
7      42
19     38
12     36
8      32
31     31
30     30
28     28
13     26
3      21
4      20
17     

,Order_Id,Customer_Id,Employee_Id,Customer_Region,Employee_Region,Region_Match,Cust_Ord_Count
0,10248,VINET,5,Western Europe,British Isles,False,5
26,10274,VINET,6,Western Europe,British Isles,False,5
47,10295,VINET,2,Western Europe,North America,False,5
471,10737,VINET,2,Western Europe,North America,False,5
473,10739,VINET,3,Western Europe,North America,False,5
1,10249,TOMSP,6,Western Europe,British Isles,False,6
185,10438,TOMSP,3,Western Europe,North America,False,6
193,10446,TOMSP,6,Western Europe,British Isles,False,6
291,10548,TOMSP,3,Western Europe,North America,False,6
350,10608,TOMSP,4,Western Europe,North America,False,6


I will now remove all duplicate entries such that there is only one row per Employee-Customer pair. This will allow me to explore patterns across customers for each employee.

In [21]:
print(df3.shape[0])
df3.drop_duplicates(subset = ['Customer_Id', 'Employee_Id'], inplace = True)
print(df3.shape[0])

801
447


### Analysis

This first analysis adresses the first part of the question:  On average across employees in each region, do customers from that region submit more orders than customers from other regions?

To do this, I will again use a fixed effects regression model. In this model, the target is the number of orders per customer, 'Cust_Ord_Count'. The main feature of interest is a dummy variable that indicates whether the customer and employee are in the same region: 'Region_Match'. I include dummy variables for each Employee in the model as well - this adjusts for the fact that some employees will tend to receive more orders overall, regardless of whether they match the customer or not. This allows for the following interpretation of the coefficient on the 'Region_Match' variable: it represents an average across all employees of the difference in order count associated with the matching of customer and employee region. This is a two-tailed test.

In [22]:
#Fixed Effects Model
fe_df = pd.get_dummies(df3, columns=['Employee_Id'], drop_first=True)
print(fe_df.columns)
emp_id_col_list = fe_df.columns[6:]
emp_id_string = '+'.join(emp_id_col_list)
f = 'Cust_Ord_Count ~ Region_Match + ' + emp_id_string
model = ols(formula=f, data=fe_df).fit()
model.summary()

Index(['Order_Id', 'Customer_Id', 'Customer_Region', 'Employee_Region',
       'Region_Match', 'Cust_Ord_Count', 'Employee_Id_2', 'Employee_Id_3',
       'Employee_Id_4', 'Employee_Id_5', 'Employee_Id_6', 'Employee_Id_7',
       'Employee_Id_8', 'Employee_Id_9'],
      dtype='object')


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Cust_Ord_Count   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7368
Date:                Tue, 19 Mar 2019   Prob (F-statistic):              0.675
Time:                        09:45:43   Log-Likelihood:                -1450.0
No. Observations:                 447   AIC:                             2920.
Df Residuals:                     437   BIC:                             2961.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               10.6849      0.807     13.232      0.000       9.098      12.272
Region_Match[T.True]     0.4501      0.805      0.559      0.576      -1.132       2.032
Employee_Id_2            0.1914      1.147      0.167      0.868      -2.063       2.446
Employee_Id_3           -0.1669      1.127     -0.148      0.882      -2.381       2.048
Employee_Id_4           -0.4051      1.082     -0.374      0.708      -2.532       1.722
Employee_Id_5            1.7276      1.431      1.207      0.228      -1.085       4.541
Employee_Id_6            0.2139      1.251      0.171      0.864      -2.246       2.673
Employee_Id_7            0.5285      1.260      0.419      0.675      -1.949       3.006
Employee_Id_8            0.5114      1.163      0.440      0.660      -1.775       2.798
Employee_Id_9            2.4944      1.409      1.771      0.077      -0.274       5.263
==============================================================================
Omnibus:                      108.876   Durbin-Watson:                   0.309
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              213.539
Skew:                           1.333   Prob(JB):                     4.27e-47
Kurtosis:                       5.087   Cond. No.                         9.15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Interpretation

The coefficient on Region_Match is 0.4501, with a P-value of 0.576. This means that we cannot reject the null hypothesis - that there is no relationship between number of orders submitted and region match between customers and employees - in favor of the alternative hypothesis that there is some such difference.

## Regional Sales Differences

### Refining the Question

In the previous analysis, we saw that employees belonged to only two regions: North America and British Isles. I was curious whether there was a difference in revenue between the two regions. If there were such a difference, this might help inform how the company could re-allocate its resources. To analyze this, I will compare the revenue per week across the two regions - this will give me sufficient sample size to have power to detect significance.

### Data Extraction

In [23]:
query4 = session.query(Order, OrderDetail, Employee).\
                filter(OrderDetail.OrderId == Order.Id).\
                filter(Employee.Id == Order.EmployeeId).\
                all()

df4 = pd.DataFrame()
for o, od, e, in query4:
    q4_df_row = pd.DataFrame({'Order_Id': o.Id, 'Order_Date':o.OrderDate, 'Employee_Region': e.Region, 
                             'Unit_Price': od.UnitPrice, 'Quantity': od.Quantity, 'Discount': od.Discount}, index = [0])
    df4 = df4.append(q4_df_row, ignore_index= True)

2019-03-19 09:45:43,464 INFO sqlalchemy.engine.base.Engine SELECT "Order"."Id" AS "Order_Id", "Order"."CustomerId" AS "Order_CustomerId", "Order"."EmployeeId" AS "Order_EmployeeId", "Order"."OrderDate" AS "Order_OrderDate", "Order"."RequiredDate" AS "Order_RequiredDate", "Order"."ShippedDate" AS "Order_ShippedDate", "Order"."ShipVia" AS "Order_ShipVia", "Order"."Freight" AS "Order_Freight", "Order"."ShipName" AS "Order_ShipName", "Order"."ShipAddress" AS "Order_ShipAddress", "Order"."ShipCity" AS "Order_ShipCity", "Order"."ShipRegion" AS "Order_ShipRegion", "Order"."ShipPostalCode" AS "Order_ShipPostalCode", "Order"."ShipCountry" AS "Order_ShipCountry", "OrderDetail"."Id" AS "OrderDetail_Id", "OrderDetail"."OrderId" AS "OrderDetail_OrderId", "OrderDetail"."ProductId" AS "OrderDetail_ProductId", "OrderDetail"."UnitPrice" AS "OrderDetail_UnitPrice", "OrderDetail"."Quantity" AS "OrderDetail_Quantity", "OrderDetail"."Discount" AS "OrderDetail_Discount", "Employee"."Id" AS "Employee_Id", "E

### Data Exploration & Cleaning

In [24]:
print(df4.info())
print(df4.describe())
df4.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 6 columns):
Order_Id           2155 non-null int64
Order_Date         2155 non-null object
Employee_Region    2155 non-null object
Unit_Price         2155 non-null object
Quantity           2155 non-null int64
Discount           2155 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 101.1+ KB
None
           Order_Id     Quantity     Discount
count   2155.000000  2155.000000  2155.000000
mean   10659.375870    23.812993     0.056167
std      241.378032    19.022047     0.083450
min    10248.000000     1.000000     0.000000
25%    10451.000000    10.000000     0.000000
50%    10657.000000    20.000000     0.000000
75%    10862.500000    30.000000     0.100000
max    11077.000000   130.000000     0.250000


,Order_Id,Order_Date,Employee_Region,Unit_Price,Quantity,Discount
0,10248,2012-07-04,British Isles,14.0000000000,12,0.0
1,10248,2012-07-04,British Isles,9.8000000000,10,0.0
2,10248,2012-07-04,British Isles,34.8000000000,5,0.0
3,10249,2012-07-05,British Isles,18.6000000000,9,0.0
4,10249,2012-07-05,British Isles,42.4000000000,40,0.0


In [25]:
#Clean Order ID Orders
df4['Order_Id'] = df4['Order_Id'].astype('category')

#Clean Employee_Region
df4['Employee_Region'] = df4['Employee_Region'].astype('category')

#Clean Order Date - convert to week category
print(df4['Order_Date'].min(), df4['Order_Date'].max())
df4['Order_Date'] = df4['Order_Date'].astype('datetime64')
df4['Order_Year_Week'] = df4['Order_Date'].dt.strftime('%Y-%U').astype('category')

#Clean Order Deatils - calculate total price
df4['Unit_Price'] = df4['Unit_Price'].astype('float64')
df4['Total_Price'] = df4['Unit_Price']*df4['Quantity']*(1 - df4['Discount'])

df4.head()

2012-07-04 2014-05-06


,Order_Id,Order_Date,Employee_Region,Unit_Price,Quantity,Discount,Order_Year_Week,Total_Price
0,10248,2012-07-04,British Isles,14.0,12,0.0,2012-27,168.0
1,10248,2012-07-04,British Isles,9.8,10,0.0,2012-27,98.0
2,10248,2012-07-04,British Isles,34.8,5,0.0,2012-27,174.0
3,10249,2012-07-05,British Isles,18.6,9,0.0,2012-27,167.4
4,10249,2012-07-05,British Isles,42.4,40,0.0,2012-27,1696.0


In [26]:
#Get total price per week/Region
price_df = df4.groupby(['Employee_Region', 'Order_Year_Week'])['Total_Price'].agg('sum').to_frame()
print(price_df.head())

#Pivot to get one row per week
price_df = price_df.unstack(level = 0).fillna(0)
price_df.head()

                                 Total_Price
Employee_Region Order_Year_Week             
British Isles   2012-27             2303.400
                2012-28             3047.120
                2012-30             2569.425
                2012-31             2146.200
                2012-32              538.600


Total_Price              
Employee_Region British Isles North America
Order_Year_Week                            
2012-27              2303.400          0.00
2012-28              3047.120       7249.36
2012-29                 0.000       5306.03
2012-30              2569.425       2106.56
2012-31              2146.200       6013.80

In [27]:
price_df.columns = price_df.columns.droplevel()
price_df.mean()

Employee_Region
British Isles    3480.623369
North America    9305.164909
dtype: float64

It looks like there is a big difference in revenue per week - let's do a formal hypothesis test

### Analysis

For this hypotheis test, I will use a paired sample t-test. I do this because there is a pair of observations from each week in our sample - one from the British Isles and one from North America. Thus, these observations are related, so a paired test is needed. The null hypothesis is that there is no difference in weekly revenue between orders processed by employees from the two regions. The alternative hypothesis is that there is a difference in weekly revenue between the two regions. This is a two-tailed test.

In [28]:
import scipy.stats as stats

stats.ttest_rel(price_df['British Isles'], price_df['North America'])

Ttest_relResult(statistic=-9.0350505597535, pvalue=1.5020048606679717e-14)

### Interpretation

The p-value is less than 0.001, so we can comfortably reject the null hypothesis and conclude that there is a difference in weekly revenue between employees from the two regions. In this case, the British Isles employees bring in significantly less revenue per week than the North America region, suggesting that that region might benefit from additional support.